In [19]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import numpy as np
import sympy as sp
# from scipy.linalg import solve_discrete_are
import matplotlib.pyplot as plt

# internal imports
from controller import controller
from nonlinear_dynamics import simulate
from mld_dynamics import mld, h
from visualizer import vis, animate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Open-loop solution

In [20]:
# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])

In [28]:
# solve in open loop
# options = {'OptimalityTol': 1e-06, 'Presolve': 0, 'Heuristics': 0}
solution, leaves, nodes, solver_time = controller.feedforward(
    x0,
    printing_period=None,
    gurobi_params={'OutputFlag': 0, 'Method': 1} # dual simplex
#     draw_label='Cart pole with walls'
)
solver_time
solution.objective

0.06925696807132983

import cProfile
cProfile.run('controller.feedforward(x0, printing_period=None)')

In [29]:
options = {'OutputFlag': 1, 'MIPGap': 0, 'Presolve': 0, 'Heuristics': 0}
x_gurobi, obj_gurobi, nodes_gurobi, solver_time = controller.feedforward_gurobi(x0, options)
nodes_gurobi, solver_time

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter MIPGap to 0.0
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Heuristics to 0.0
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 906 rows, 224 columns and 2684 nonzeros
Model fingerprint: 0x6ba9a65b
Model has 110 quadratic objective terms
Variable types: 144 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [6e-04, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-03, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+02]
Variable types: 140 continuous, 84 integer (80 binary)

Root relaxation: objective 4.265780e-02, 233 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexp

(221, 0.12791085243225098)

In [ ]:
vis.jupyter_cell()

In [ ]:
animate(solution.variables['x'], h)

## Solve with and without warm start

In [18]:
e0 = np.random.randn(mld.nx) * 0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
x1 = solution.variables['x'][1]
# warm_start, ws_time = controller.construct_warm_start(leaves, x0, uc0, ub0, e0, x1)
_, ws_time = controller.construct_warm_start(leaves, x0, uc0, ub0, e0, x1)
ws_time

0.014200210571289062

import cProfile
e0 = np.random.randn(mld.nx) * 0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
cProfile.run('controller.construct_warm_start(leaves, x0, uc0, ub0, e0)')

In [ ]:
u0 = np.concatenate((uc0, ub0))
x1 = mld.A.dot(x0) + mld.B.dot(u0) + e0
solution, leaves, nodes, solver_time = controller.feedforward(
    x1,
    warm_start=warm_start,
    gurobi_options={'Method': 1}, # dual simplex
#     draw_label='Cart pole with walls warm start'
)
solver_time

In [ ]:
solution, leaves, nodes, solver_time = controller.feedforward(x1)
solver_time

## Solve in closed loop

In [ ]:
from visualizer import vis, visualize
vis.jupyter_cell()

In [ ]:
# np.random.seed(1)

# set up simulation
T_sim = 60
x_true = [x0]
x_sim = []
x_predicted = []
u_sim = []
warm_start = None

for t in range(T_sim):
    print('Time step %d.'%t)
    
    # solve miqp
    solution, leaves = controller.feedforward(
        x_true[-1],
        warm_start=warm_start,
#         printing_period=None
    )[:2]
    
    # reorganize solution
    uc0 = solution.variables['uc'][0]
    ub0 = solution.variables['ub'][0]
    u0 = np.concatenate((uc0, ub0))
    x_sim_t = simulate(x_true[-1], h, uc0[0])
    x1 = x_sim_t[-1]
#     x1 = solution.variables['x'][1]
    e0 = x1 - mld.A.dot(x_true[-1]) - mld.B.dot(u0)
    print('predicted state:', solution.variables['x'][1])
    print('true state:', x1)
    print('modeling error:', e0)
#     visualize(x1)
    
    # generate warm start
    warm_start = controller.construct_warm_start(leaves, x_true[-1], uc0, ub0, e0)
    
    # retrieve closed-loop trajectory
#     x_sim.append(x1)
    u_sim.append(uc0)
    
    # retrieve closed-loop trajectory
    x_predicted.append(solution.variables['x'][1])
    x_true.append(x1)
    x_sim.extend(x_sim_t)

In [ ]:
indices =  [0,1]
fig = plt.figure()

plt.plot(
    [xt[indices[0]] for xt in x_sim],
    [xt[indices[1]] for xt in x_sim],
    color='c',
    label='Closed-loop trajectory',
    linewidth=3
)

for t in range(T_sim):
    label = 'Next-state prediction'
    if t > 0:
        label=None
    plt.plot(
        [x_true[t][indices[0]], x_predicted[t][indices[0]]],
        [x_true[t][indices[1]], x_predicted[t][indices[1]]],
        c='b',
        label=label
    )
    plt.scatter(
        [x_predicted[t][indices[0]]],
        [x_predicted[t][indices[1]]],
        c='b',
        marker='o',
        s=5,
        zorder=3
    )
plt.xlabel(r'$x_%d$'%(indices[0]+1))
plt.ylabel(r'$x_%d$'%(indices[1]+1))
# ax.set_zlabel(r'Solved QPs')
plt.legend()
plt.grid(True)
# plt.show()
plt.savefig('error_soft.pdf')

## Animation

In [ ]:
from visualizer import vis, animate
vis.jupyter_cell()

In [ ]:
animate(x_sim, h)